In [2]:
import scanpy as sc
import re

In [222]:
adata = sc.read("../../data/adamson.h5ad")

# print(adata.obs["UMI count"].head())
# print(adata.var)

In [223]:
adata.obs["perturbation"].value_counts()

perturbation
3x_neg_ctrl_pMJ144-1     1715
ATF6_PERK_IRE1_pMJ158    1693
ATF6_only_pMJ145         1691
IRE1_only_pMJ148         1685
3x_neg_ctrl_pMJ144-2     1650
PERK_IRE1_pMJ154         1629
ATF6_PERK_pMJ150         1603
PERK_only_pMJ146         1560
ATF6_IRE1_pMJ152         1449
*                          13
PSMD12_pDS009               6
PSMA1_pDS007                4
IER3IP1_pDS003              3
C7orf26_pDS004              2
XBP1_pBA578                 2
Gal4-4(mod)_pBA582          1
ATF4_pBA576                 1
SNAI1_pDS266                1
XBP1_pBA579                 1
YIPF5_pDS001                1
Name: count, dtype: int64

In [62]:
adata

AnnData object with n_obs × n_vars = 15006 × 32738
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [86]:
# Feature selection for highly variable genes, feel free to modify parameters

sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=2000, layer="counts")

In [87]:
adata = adata[:,adata.var["highly_variable"]]

In [65]:
adata

View of AnnData object with n_obs × n_vars = 15006 × 2000
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'n_genes'
    var: 'ensembl_id', 'ncounts', 'ncells', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'log1p', 'hvg'
    layers: 'counts'

In [88]:
adata = adata[~adata.obs["perturbation"].isna() & (adata.obs["perturbation"] != "*")].copy()
# adata = adata[~adata.obs["perturbation"].str.contains(r"ctrl|control", flags=re.IGNORECASE, na=False)].copy()
adata.obs["perturbation"].unique()

['3x_neg_ctrl_pMJ144-1', '3x_neg_ctrl_pMJ144-2', 'ATF6_PERK_IRE1_pMJ158', 'ATF6_PERK_pMJ150', 'ATF6_only_pMJ145', ..., 'ATF4_pBA576', 'SNAI1_pDS266', 'XBP1_pBA579', 'Gal4-4(mod)_pBA582', 'YIPF5_pDS001']
Length: 19
Categories (19, object): ['3x_neg_ctrl_pMJ144-1', '3x_neg_ctrl_pMJ144-2', 'ATF4_pBA576', 'ATF6_IRE1_pMJ152', ..., 'SNAI1_pDS266', 'XBP1_pBA578', 'XBP1_pBA579', 'YIPF5_pDS001']

In [224]:
%run ../utils.py
split = {"train": 0.7, "val": 0.15, "test": 0.15}
assign_splits_proportional(adata, split)

chose PERK_only_pMJ146, so now test_perts_count=1560
chose ATF6_PERK_pMJ150, so now test_perts_count=3163


In [234]:
print(adata.obs["split"].value_counts())
print("perts in train", list(adata.obs.loc[adata.obs["split"] == "train", "perturbation"].unique()))
print("perts in test", list(adata.obs.loc[adata.obs["split"] == "test", "perturbation"].unique()))
print("perts in val", list(adata.obs.loc[adata.obs["split"] == "val", "perturbation"].unique()))

split
train    9502
test     3163
val      2341
Name: count, dtype: int64
perts in train ['3x_neg_ctrl_pMJ144-1', '3x_neg_ctrl_pMJ144-2', 'ATF6_PERK_IRE1_pMJ158', 'ATF6_only_pMJ145', 'PERK_IRE1_pMJ154', 'ATF6_IRE1_pMJ152', 'IRE1_only_pMJ148', '*', nan, 'IER3IP1_pDS003', 'PSMA1_pDS007', 'PSMD12_pDS009', 'C7orf26_pDS004', 'ATF4_pBA576', 'SNAI1_pDS266', 'XBP1_pBA579', 'Gal4-4(mod)_pBA582', 'YIPF5_pDS001']
perts in test ['ATF6_PERK_pMJ150', 'PERK_only_pMJ146']
perts in val ['ATF6_IRE1_pMJ152', 'ATF6_only_pMJ145', 'ATF6_PERK_IRE1_pMJ158', 'IRE1_only_pMJ148', 'PERK_IRE1_pMJ154', nan, 'XBP1_pBA578', '*', 'PSMA1_pDS007', 'PSMD12_pDS009']


In [69]:
if "log1p" in adata.uns:
    del adata.uns["log1p"]
adata

AnnData object with n_obs × n_vars = 11332 × 2000
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'n_genes', 'split'
    var: 'ensembl_id', 'ncounts', 'ncells', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'
    layers: 'counts'

In [99]:
perts = list(adata.obs["perturbation"].unique())
pert_map = {}
for pert in perts:
    pert_split = [gene for gene in pert.split("_")[:-1] if gene != 'only']
    pert_map[pert] = pert_split
print(pert_map)

{'3x_neg_ctrl_pMJ144-1': ['3x', 'neg', 'ctrl'], '3x_neg_ctrl_pMJ144-2': ['3x', 'neg', 'ctrl'], 'ATF6_PERK_IRE1_pMJ158': ['ATF6', 'PERK', 'IRE1'], 'ATF6_PERK_pMJ150': ['ATF6', 'PERK'], 'ATF6_only_pMJ145': ['ATF6'], 'PERK_IRE1_pMJ154': ['PERK', 'IRE1'], 'ATF6_IRE1_pMJ152': ['ATF6', 'IRE1'], 'IRE1_only_pMJ148': ['IRE1'], 'PERK_only_pMJ146': ['PERK'], 'XBP1_pBA578': ['XBP1'], 'IER3IP1_pDS003': ['IER3IP1'], 'PSMA1_pDS007': ['PSMA1'], 'PSMD12_pDS009': ['PSMD12'], 'C7orf26_pDS004': ['C7orf26'], 'ATF4_pBA576': ['ATF4'], 'SNAI1_pDS266': ['SNAI1'], 'XBP1_pBA579': ['XBP1'], 'Gal4-4(mod)_pBA582': ['Gal4-4(mod)'], 'YIPF5_pDS001': ['YIPF5']}


In [74]:
# adata = sc.read("../../data/preprocessed/adamson_preprocessed.h5ad")
adata.obs["perturbation"].value_counts()

perturbation
ATF6_PERK_IRE1_pMJ158    1693
ATF6_only_pMJ145         1691
IRE1_only_pMJ148         1685
PERK_IRE1_pMJ154         1629
ATF6_PERK_pMJ150         1603
PERK_only_pMJ146         1560
ATF6_IRE1_pMJ152         1449
PSMD12_pDS009               6
PSMA1_pDS007                4
IER3IP1_pDS003              3
XBP1_pBA578                 2
C7orf26_pDS004              2
XBP1_pBA579                 1
ATF4_pBA576                 1
SNAI1_pDS266                1
Gal4-4(mod)_pBA582          1
YIPF5_pDS001                1
Name: count, dtype: int64

In [82]:
adata = sc.read("../../data/predictions/adamson_pred_baseline_0.h5ad")

In [83]:
import sys
from pathlib import Path

# add the parent directory of the notebook to sys.path
sys.path.append(str(Path().resolve().parent))

from utils import compute_degs

In [84]:
%run ../utils.py

In [85]:
# adata.obs["perturbation"].value_counts()
curr_deg_results = compute_degs(adata)

Computing DEGs vsrest: 100%|██████████| 8/8 [00:00<00:00, 1799.84it/s]


In [86]:
# adata.uns["rank_genes_groups_vsrest"]
curr_deg_results.keys()

dict_keys(['params', 'names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges'])

In [87]:
import numpy as np
import pandas as pd

names_df_vsrest = pd.DataFrame(curr_deg_results["names"])
scores_df_vsrest = pd.DataFrame(curr_deg_results["scores"])

pert_normalized_abs_scores_vsrest = {}
for pert in scores_df_vsrest.columns:
    if pert == 'control': # Typically no scores for control in vsrest, but good to check
        continue

    abs_scores = np.abs(scores_df_vsrest[pert].values) # Ensure it's a numpy array
    min_val = np.min(abs_scores)
    max_val = np.max(abs_scores)

    if max_val == min_val:
        if max_val == 0: # All scores are 0
            normalized_weights = np.zeros_like(abs_scores)
        else: # All scores are the same non-zero value
            # Squaring ones will still be ones, which is fine.
            normalized_weights = np.ones_like(abs_scores) 
    else:
        normalized_weights = (abs_scores - min_val) / (max_val - min_val)

    # Ensure no NaNs in weights, replace with 0 if any (e.g. if a gene had NaN score originally)
    normalized_weights = np.nan_to_num(normalized_weights, nan=0.0)

    # Make weighting stronger by squaring the normalized weights
    stronger_normalized_weights = np.square(normalized_weights)

    weights = pd.Series(stronger_normalized_weights, index=names_df_vsrest[pert].values, name=pert)
    # Order by the var_names
    weights = weights.reindex(adata.var_names)
    pert_normalized_abs_scores_vsrest[pert] = weights

In [110]:
example_pert = "PERK_only_pMJ146"
print(pert_normalized_abs_scores_vsrest.keys())
weights = pert_normalized_abs_scores_vsrest[example_pert]
print("og weights", weights)
weights_total = weights.sum()
if weights_total > 0:
    weights /= weights_total
sum(weights)

dict_keys(['ATF6_only_pMJ145', 'C7orf26_pDS004', 'IER3IP1_pDS003', 'IRE1_only_pMJ148', 'PERK_only_pMJ146', 'PSMA1_pDS007', 'PSMD12_pDS009', 'XBP1_pBA578'])
og weights gene_symbol
HES4           0.000193
ISG15          0.000007
AGRN           0.000005
RP1-140A9.1    0.000004
RP5-892K4.1    0.000046
                 ...   
MT-ND3         0.000062
MT-ND4L        0.000114
MT-ND4         0.000559
MT-ND6         0.000625
MT-CYB         0.000163
Name: PERK_only_pMJ146, Length: 2000, dtype: float32


0.999999960212826

In [159]:
# get DEGs for this perturbation
# paper computes r2 both with degs only using pert_degs_idx mask, and with all genes. which should i do? probably use mask
pert_degs = list(set(adata.uns['deg_dict_vsrest'][example_pert]['up']) | 
                    set(adata.uns['deg_dict_vsrest'][example_pert]['down']))
pert_degs_idx = adata.var_names.isin(pert_degs)
# TODO ^^^

# get deltas
def is_control(pert):
    return re.search(r"ctrl|control", pert, flags=re.IGNORECASE) is not None

perturbation = adata.obs["perturbation"].astype(str)
pert_mask = ~perturbation.apply(is_control).values

mu_all = adata.X[pert_mask].mean(axis=0)

def pseudobulk_means(adata, pert, pert_mask, pred_layer):
    mask = (adata.obs["perturbation"] == pert).values & pert_mask

    mu_p = adata.X[mask].mean(axis=0)
    mu_p_hat = adata.layers[pred_layer][mask].mean(axis=0)

    return mu_p, mu_p_hat

mu_p, mu_p_hat = pseudobulk_means(adata, example_pert, pert_mask, "pred_baseline_mean")

delta = np.array(mu_p - mu_all).squeeze()
delta_hat = np.array(mu_p_hat - mu_all).squeeze()
print(delta, delta_hat)

[-0.03739727  0.00730634  0.00280336 ... -0.0537796   0.08661062
 -0.02866697] [-0.00335258 -0.01381677 -0.00080636 ...  0.02323723 -0.02974999
  0.01058769]


In [161]:
from sklearn.metrics import r2_score
def r2_score_on_deltas(delta_true, delta_pred, weights):
    if len(delta_true) < 2 or len(delta_pred) < 2 or delta_true.shape != delta_pred.shape:
        return np.nan
    return r2_score(delta_true, delta_pred, sample_weight=weights)

score = r2_score_on_deltas(delta, delta_hat, weights)
print(score)
adata

-1.3527298849566258


AnnData object with n_obs × n_vars = 8323 × 2000
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'n_genes', 'split'
    var: 'ensembl_id', 'ncounts', 'ncells', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg', 'deg_dict_vsrest', 'rank_genes_groups_vsrest'
    layers: 'counts', 'pred_baseline_mean'

In [162]:
unique_perts = adata.obs.loc[adata.obs["split"] == "test", "perturbation"].unique().tolist()
print(unique_perts)

['PERK_only_pMJ146']
